# 재귀함수 설계

이 노트에서 우리는 재귀함수를 어떻게 설계하는지 배우고, 이를 이용해 몇 가지 정렬 알고리즘을 구현한다.

In [6]:
"""
재귀 함수의 콜스택을 시각화하는 데코레이터
"""
import sys
from io import StringIO
from functools import wraps


class CallStackVisualizer:
    def __init__(self):
        self.depth = 0
        self.original_stdout = sys.stdout

    def get_indent(self):
        """현재 깊이에 맞는 들여쓰기 문자열 반환"""
        return "|   " * self.depth

    def visualize(self, func):
        """재귀 함수를 시각화하는 데코레이터"""
        @wraps(func)
        def wrapper(*args, **kwargs):
            # 함수 호출 시작
            indent = self.get_indent()

            # 함수명과 파라미터 출력
            args_str = ", ".join(repr(arg) for arg in args)
            if kwargs:
                kwargs_str = ", ".join(f"{k}={v!r}" for k, v in kwargs.items())
                params = f"{args_str}, {kwargs_str}" if args_str else kwargs_str
            else:
                params = args_str

            print(f"{indent}{func.__name__}({params})", file=self.original_stdout)

            # 깊이 증가
            self.depth += 1

            # stdout 리다이렉션 설정
            original_stdout = sys.stdout
            captured_output = StringIO()

            class IndentedStdout:
                def __init__(self, original, depth_getter):
                    self.original = original
                    self.depth_getter = depth_getter
                    self.buffer = []  # 한 줄씩 모으기 위한 버퍼

                def write(self, text):
                    if not text:
                        return

                    # 개행 문자가 나올 때까지 버퍼에 모음
                    if '\n' in text:
                        parts = text.split('\n')
                        # 마지막 부분 전까지 처리
                        for i, part in enumerate(parts[:-1]):
                            self.buffer.append(part)
                            line = ''.join(self.buffer)
                            if line:  # 빈 줄이 아닌 경우만
                                indent = "|   " * self.depth_getter()
                                self.original.write(f'{indent}"{line}"\n')
                            self.buffer = []
                        # 마지막 부분은 버퍼에 보관
                        if parts[-1]:
                            self.buffer.append(parts[-1])
                    else:
                        self.buffer.append(text)

                def flush(self):
                    # 버퍼에 남아있는 내용 출력
                    if self.buffer:
                        line = ''.join(self.buffer)
                        if line:
                            indent = "|   " * self.depth_getter()
                            self.original.write(f'{indent}"{line}"\n')
                        self.buffer = []
                    self.original.flush()

            sys.stdout = IndentedStdout(self.original_stdout, lambda: self.depth)

            try:
                # 원본 함수 실행
                result = func(*args, **kwargs)
            finally:
                # stdout 복원
                sys.stdout = original_stdout

                # 깊이 감소
                self.depth -= 1

                # 반환값 출력 (None이 아닌 경우만)
                if result is not None:
                    indent = self.get_indent()
                    print(f"{indent}returned {result!r}", file=self.original_stdout)

            return result

        return wrapper


# 전역 visualizer 인스턴스
_visualizer = CallStackVisualizer()

def visualize_callstack(func):
    """
    재귀 함수의 콜스택을 시각화하는 데코레이터

    사용법:
        @visualize_callstack
        def f(n, s, t, w):
            if n >= 1:
                f(n-1, s, w, t)
                print(n, s, t)
                f(n-1, w, t, s)

        f(3, 'A', 'C', 'B')
    """
    return _visualizer.visualize(func)

## 하노이 탑과 재귀함수 설계

1. 반복되는 동작 찾기

하노이 탑에서 반복되는 동작은 세 개의 기둥 A, B, C 사이에서 k개의 원판을 "전부" 옮기는 것이다.

일반적으로는 이것만으로도 재귀함수를 작성하기 충분하다.

그러나 하노이탑의 경우 n개를 (시작, 종료, 경유)로 옮겨야 할 때, 그 안에서 n-1개를 '시작'에서 '경유'로 옮기고, '종료'가 경유지로 변하는 교대 현상을 구현해야 하기 때문에, 파라메터가 중간에 적절하게 교체되어야 한다.

- `f(n, 'A', 'C', 'B')`는 n개의 원판 전부를 A에서 C로 옮기는 함수이다. 이때 n-1개 전부는 A에서 B로 옮겨져야 한다.
- `f(n-1, 'A', 'B', 'C')`는 n-1개의 원판 전부를 A에서 B로 옮기는 함수이다. 이때 n-2개 전부는 A에서 C로 옮겨져야 한다.
- `f(n-2, 'A', 'C', 'B')`는 n-2개의 원판 전부를 A에서 C로 옮기는 함수이다. 이때 n-3개 전부는 A에서 B로 옮겨져야 한다.

2. 기저 조건 찾기

n=1일 때 == 시작지점 원판이 하나 남은 상태

이때의 원판은 그냥 목표지점으로 옮기면 된다. (1, s, t, w)인 경우 print(1, s, t)를 출력.

n<1일 때 == 아무 동작 하지 않음

In [1]:
from callstack_visualizer import visualize_callstack

@visualize_callstack
def f(n, s, t, w):
    if n > 1:
        f(n-1, s, w, t)
        print(n, s, t)
        f(n-1, w, t, s)

    else:
        print(n, s, t)

f(3, 'A', 'C', 'B')

f(3, 'A', 'C', 'B')
|   f(2, 'A', 'B', 'C')
|   |   f(1, 'A', 'C', 'B')
|   |   |   "1 A C"
|   |   "2 A B"
|   |   f(1, 'C', 'B', 'A')
|   |   |   "1 C B"
|   "3 A C"
|   f(2, 'B', 'C', 'A')
|   |   f(1, 'B', 'A', 'C')
|   |   |   "1 B A"
|   |   "2 B C"
|   |   f(1, 'A', 'C', 'B')
|   |   |   "1 A C"


## 퀵 정렬과 재귀함수 설계

1. 반복되는 동작 찾기

퀵 정렬에서 반복되는 동작은 배열을 피벗(pivot) 기준으로 분할하고, 분할된 부분을 각각 정렬하는 것이다.

- 피벗보다 작은 값들 → 왼쪽 그룹
- 피벗과 같은 값들 → 중간 그룹  
- 피벗보다 큰 값들 → 오른쪽 그룹

분할 후, 왼쪽과 오른쪽 그룹에 대해 같은 과정을 재귀적으로 반복한다.

최종 결과는 `정렬된 왼쪽 + 중간 + 정렬된 오른쪽`을 합친 것이다.

2. 기저 조건 찾기

`len(arr) <= 1`일 때 == 배열의 원소가 0개 또는 1개

이 경우는 이미 정렬되어 있으므로 그대로 반환한다.

In [2]:
# 퀵 정렬 (Quick Sort)
@visualize_callstack
def quick_sort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    print(f"left: {left}, middle: {middle}, right: {right}")
    return quick_sort(left) + middle + quick_sort(right)

print("=== 퀵 정렬 ===")
result = quick_sort([3, 6, 8, 10, 1, 2, 1])
print(f"\n정렬 결과: {result}")

=== 퀵 정렬 ===
quick_sort([3, 6, 8, 10, 1, 2, 1])
|   "left: [3, 6, 8, 1, 2, 1], middle: [10], right: []"
|   quick_sort([3, 6, 8, 1, 2, 1])
|   |   "left: [], middle: [1, 1], right: [3, 6, 8, 2]"
|   |   quick_sort([])
|   |   returned []
|   |   quick_sort([3, 6, 8, 2])
|   |   |   "left: [3, 6, 2], middle: [8], right: []"
|   |   |   quick_sort([3, 6, 2])
|   |   |   |   "left: [3, 2], middle: [6], right: []"
|   |   |   |   quick_sort([3, 2])
|   |   |   |   |   "left: [], middle: [2], right: [3]"
|   |   |   |   |   quick_sort([])
|   |   |   |   |   returned []
|   |   |   |   |   quick_sort([3])
|   |   |   |   |   returned [3]
|   |   |   |   returned [2, 3]
|   |   |   |   quick_sort([])
|   |   |   |   returned []
|   |   |   returned [2, 3, 6]
|   |   |   quick_sort([])
|   |   |   returned []
|   |   returned [2, 3, 6, 8]
|   returned [1, 1, 2, 3, 6, 8]
|   quick_sort([])
|   returned []
returned [1, 1, 2, 3, 6, 8, 10]

정렬 결과: [1, 1, 2, 3, 6, 8, 10]


## 선택 정렬과 재귀함수 설계

1. 반복되는 동작 찾기

선택 정렬에서 반복되는 동작은 정렬되지 않은 부분에서 최솟값을 찾아 맨 앞으로 이동시키는 것이다.

- `start` 위치부터 끝까지 중에서 최솟값을 찾는다
- 최솟값을 `start` 위치와 교환한다
- `start+1` 위치부터 같은 과정을 반복한다

이 과정을 재귀적으로 반복하면, 앞에서부터 차례대로 정렬된다.

2. 기저 조건 찾기

`start >= len(arr) - 1`일 때 == 정렬할 원소가 1개 이하 남음

이 경우 더 이상 정렬할 필요가 없으므로 배열을 그대로 반환한다.

In [3]:
# 선택 정렬 (Selection Sort)
@visualize_callstack
def selection_sort(arr, start=0):
    if start >= len(arr) - 1:
        return arr

    # 최솟값 찾기
    min_idx = start
    for i in range(start + 1, len(arr)):
        if arr[i] < arr[min_idx]:
            min_idx = i

    # 교환
    print(f"교환 전: {arr}, start: {start}, min_idx: {min_idx}")
    arr[start], arr[min_idx] = arr[min_idx], arr[start]
    print(f"교환 후: {arr}")

    return selection_sort(arr, start + 1)

print("=== 선택 정렬 ===")
result = selection_sort([64, 25, 12, 22, 11])
print(f"\n정렬 결과: {result}")

=== 선택 정렬 ===
selection_sort([64, 25, 12, 22, 11])
|   "교환 전: [64, 25, 12, 22, 11], start: 0, min_idx: 4"
|   "교환 후: [11, 25, 12, 22, 64]"
|   selection_sort([11, 25, 12, 22, 64], 1)
|   |   "교환 전: [11, 25, 12, 22, 64], start: 1, min_idx: 2"
|   |   "교환 후: [11, 12, 25, 22, 64]"
|   |   selection_sort([11, 12, 25, 22, 64], 2)
|   |   |   "교환 전: [11, 12, 25, 22, 64], start: 2, min_idx: 3"
|   |   |   "교환 후: [11, 12, 22, 25, 64]"
|   |   |   selection_sort([11, 12, 22, 25, 64], 3)
|   |   |   |   "교환 전: [11, 12, 22, 25, 64], start: 3, min_idx: 3"
|   |   |   |   "교환 후: [11, 12, 22, 25, 64]"
|   |   |   |   selection_sort([11, 12, 22, 25, 64], 4)
|   |   |   |   returned [11, 12, 22, 25, 64]
|   |   |   returned [11, 12, 22, 25, 64]
|   |   returned [11, 12, 22, 25, 64]
|   returned [11, 12, 22, 25, 64]
returned [11, 12, 22, 25, 64]

정렬 결과: [11, 12, 22, 25, 64]


## 버블 정렬과 재귀함수 설계

1. 반복되는 동작 찾기

버블 정렬에서 반복되는 동작은 인접한 원소들을 비교하며 한 번의 패스(pass)를 수행하는 것이다.

- 한 번의 패스에서 인접한 두 원소를 비교하여 큰 값을 뒤로 보낸다
- 패스가 끝나면 가장 큰 값이 맨 뒤로 이동한다
- 나머지 부분(맨 뒤 제외)에 대해 같은 과정을 반복한다

매 재귀마다 정렬해야 할 범위(`n`)가 1씩 줄어든다.

2. 기저 조건 찾기

`n <= 1`일 때 == 정렬할 원소가 1개 이하

이 경우 이미 정렬되어 있으므로 배열을 그대로 반환한다.

In [4]:
# 버블 정렬 (Bubble Sort) - 재귀 버전
@visualize_callstack
def bubble_sort(arr, n=None):
    if n is None:
        n = len(arr)

    # 기저 조건: 정렬할 원소가 1개 이하
    if n <= 1:
        return arr

    # 한 번의 패스: 가장 큰 값을 맨 뒤로
    for i in range(n - 1):
        if arr[i] > arr[i + 1]:
            arr[i], arr[i + 1] = arr[i + 1], arr[i]

    print(f"패스 완료: {arr}, 정렬된 부분: 마지막 {len(arr) - n + 1}개")

    # 나머지 부분 재귀 정렬
    return bubble_sort(arr, n - 1)

print("=== 버블 정렬 ===")
result = bubble_sort([64, 34, 25, 12, 22, 11, 90])
print(f"\n정렬 결과: {result}")

=== 버블 정렬 ===
bubble_sort([64, 34, 25, 12, 22, 11, 90])
|   "패스 완료: [34, 25, 12, 22, 11, 64, 90], 정렬된 부분: 마지막 1개"
|   bubble_sort([34, 25, 12, 22, 11, 64, 90], 6)
|   |   "패스 완료: [25, 12, 22, 11, 34, 64, 90], 정렬된 부분: 마지막 2개"
|   |   bubble_sort([25, 12, 22, 11, 34, 64, 90], 5)
|   |   |   "패스 완료: [12, 22, 11, 25, 34, 64, 90], 정렬된 부분: 마지막 3개"
|   |   |   bubble_sort([12, 22, 11, 25, 34, 64, 90], 4)
|   |   |   |   "패스 완료: [12, 11, 22, 25, 34, 64, 90], 정렬된 부분: 마지막 4개"
|   |   |   |   bubble_sort([12, 11, 22, 25, 34, 64, 90], 3)
|   |   |   |   |   "패스 완료: [11, 12, 22, 25, 34, 64, 90], 정렬된 부분: 마지막 5개"
|   |   |   |   |   bubble_sort([11, 12, 22, 25, 34, 64, 90], 2)
|   |   |   |   |   |   "패스 완료: [11, 12, 22, 25, 34, 64, 90], 정렬된 부분: 마지막 6개"
|   |   |   |   |   |   bubble_sort([11, 12, 22, 25, 34, 64, 90], 1)
|   |   |   |   |   |   returned [11, 12, 22, 25, 34, 64, 90]
|   |   |   |   |   returned [11, 12, 22, 25, 34, 64, 90]
|   |   |   |   returned [11, 12, 22, 25, 34, 64, 90]
|   |   

## 삽입 정렬과 재귀함수 설계

1. 반복되는 동작 찾기

삽입 정렬에서 반복되는 동작은 배열의 앞부분을 정렬한 후, 다음 원소를 적절한 위치에 삽입하는 것이다.

- 먼저 `n-1`개의 원소를 재귀적으로 정렬한다
- `n`번째 원소(마지막 원소)를 정렬된 부분의 적절한 위치에 삽입한다
- 삽입 위치는 앞에서부터 비교하며 찾는다

이 과정은 "작은 부분을 먼저 정렬하고, 새 원소를 추가"하는 방식이다.

2. 기저 조건 찾기

`n <= 1`일 때 == 배열의 원소가 1개 이하

원소가 1개 이하면 이미 정렬되어 있으므로 그대로 반환한다.

In [5]:
# 삽입 정렬 (Insertion Sort) - 재귀 버전
@visualize_callstack
def insertion_sort(arr, n=None):
    if n is None:
        n = len(arr)

    # 기저 조건: 원소가 1개면 이미 정렬됨
    if n <= 1:
        return arr

    # 먼저 n-1개 정렬
    insertion_sort(arr, n - 1)

    # 마지막 원소를 적절한 위치에 삽입
    last = arr[n - 1]
    j = n - 2

    print(f"삽입 전: {arr}, 삽입할 값: {last}")

    # 삽입할 위치 찾기
    while j >= 0 and arr[j] > last:
        arr[j + 1] = arr[j]
        j -= 1

    arr[j + 1] = last
    print(f"삽입 후: {arr}")

    return arr

print("=== 삽입 정렬 ===")
result = insertion_sort([12, 11, 13, 5, 6])
print(f"\n정렬 결과: {result}")

=== 삽입 정렬 ===
insertion_sort([12, 11, 13, 5, 6])
|   insertion_sort([12, 11, 13, 5, 6], 4)
|   |   insertion_sort([12, 11, 13, 5, 6], 3)
|   |   |   insertion_sort([12, 11, 13, 5, 6], 2)
|   |   |   |   insertion_sort([12, 11, 13, 5, 6], 1)
|   |   |   |   returned [12, 11, 13, 5, 6]
|   |   |   |   "삽입 전: [12, 11, 13, 5, 6], 삽입할 값: 11"
|   |   |   |   "삽입 후: [11, 12, 13, 5, 6]"
|   |   |   returned [11, 12, 13, 5, 6]
|   |   |   "삽입 전: [11, 12, 13, 5, 6], 삽입할 값: 13"
|   |   |   "삽입 후: [11, 12, 13, 5, 6]"
|   |   returned [11, 12, 13, 5, 6]
|   |   "삽입 전: [11, 12, 13, 5, 6], 삽입할 값: 5"
|   |   "삽입 후: [5, 11, 12, 13, 6]"
|   returned [5, 11, 12, 13, 6]
|   "삽입 전: [5, 11, 12, 13, 6], 삽입할 값: 6"
|   "삽입 후: [5, 6, 11, 12, 13]"
returned [5, 6, 11, 12, 13]

정렬 결과: [5, 6, 11, 12, 13]
